# Задание 1 - двухточечная квадратура Гаусса

In [155]:
import numpy as np

In [156]:
from scipy.special import roots_legendre

def gauss_2(a, b):
    r"""Return nodes and weights for a two-point Gauss quadrature on [a, b].
    
    Parameters
    ----------
    a, b : float
       Boundaries of the integration interval
       
    Returns
    -------
    x1, x2, w1, w2 : float
       Nodes and weights of the quadrature.
    """
    
    x,w = roots_legendre(2)

    x_ab = (b+a)/2+(b-a)/2*x # Переводим из отрезка [-1,1] в заданный [a,b]

    w_ab = w*(b-a)/2 # Учёт множителя перед интегралом при замене переменных

    return x_ab[0], x_ab[1], w_ab[0], w_ab[1]



In [157]:
from numpy.testing import assert_allclose

x1, x2, w1, w2 = gauss_2(0, 1)

def f(x, n): 
    return x**n

for n in [0, 1, 2, 3]:
    assert_allclose(w1*f(x1, n=n) + w2*f(x2, n=n),
                    1./(n+1), atol=1e-14)

# Задание 2 - Вычитание интегрируемых сингулярностей

Вычтем сингулярности: для этого нужно вычесть $1$ при $x=0$ и $e$ при $x=1$, тогда в числителе останется функция $\propto x$ (или $x-1$ соответственно), а так как в знаменателе в обоих случаях особенности порядка $x^{1/2}$ ($(1-x)^{1/2}$ соответственно), то сингулярности пропадут.

$$I = \int_0^1\dfrac{e^x}{\sqrt{x(1-x)}}dx = \int_0^{1}\dfrac{e^x-1-(e-1)x}{\sqrt{x(1-x)}}dx + \int_{0}^{1}\dfrac{1+(e-1)x}{\sqrt{x(1-x)}}dx = $$

Используем известные значения интегралов, данных по условию: $$= \int_0^{1}\dfrac{e^x-1-(e-1)x}{\sqrt{x(1-x)}}dx + \pi +(e-1)\dfrac{\pi}{2}$$

In [158]:
def func(x):
    return (np.exp(x)-1-(np.exp(1)-1)*x)/(np.sqrt(x*(1-x)))


def integ(npts=10):
    """Compute the value of the integral above.
    
    Subtract the singularities and use the trapezoid rule. 
    
    Parameters
    ----------
    npts : int
        The number of points for the trapezoid rule
        
    Returns
    -------
    I : float
       The computed value of the integral
    """

    xs = np.linspace(0,1, npts)

    return 1/npts*np.sum(func(xs[1:-1])) # не берём крайние точки, так как по построению в них значение функции = 0


In [159]:
ans = integ(10) + np.pi +(np.exp(1)-1)*np.pi/2

ans

5.553465684476267

Аналитически погрешность интеграла оценить не получится, так как для правила трапеций соответствующая формула $$\Delta I = \dfrac{(b-a)h^2}{12}M_2$$ где $M_2=max_{x\in[a,b]}|f''(x)|$, а в нашем случае вторая производная на отрезке $[0,1]$ неограничена.

Оценим по правилу Рунге:

In [160]:
delta = abs((integ(10)-integ(5))/(2**2-1))

delta

0.0188318147086944

Более точное значение интеграла (вычислили с помощью Вольфрама) равно $\approx 5.50843$. Попробуем увеличить количество точек для формул трапеции и посмотрим на значение интеграла:

Также заметим, что это значение не попало в интервал погрешности, посчитанной с помощью метода Рунге при $n=10$

In [161]:
def runge_checks(n, a, b, eps=1e-12):
    Ih = integ(n)+ np.pi +(np.exp(1)-1)*np.pi/2
    Dh = None
    while True:
        h_2 = (b-a) / (2*n)
        Ih_2 = integ(2*n)+ np.pi +(np.exp(1)-1)*np.pi/2
        Dh_2 = (Ih - Ih_2) / (2**2 - 1) # p = 2 для Симпсона
        Ch_2 = Dh_2 / h_2**2; ps = np.log2(Dh / Dh_2) if Dh != None else np.nan
        print('I(h) = %.16f, err(h) = %.6e, p* = %4.2f, C = %.6e' % \
              (Ih_2, Dh_2, ps, Ch_2))
        if abs(Dh_2) < eps:
            break
        n *= 2; Ih = Ih_2; Dh = Dh_2
        if n > 10000: print('Too large n'); break
    return Ih_2

In [162]:
runge_checks(5,0,1)

I(h) = 5.5534656844762669, err(h) = 1.883181e-02, p* =  nan, C = 1.883181e+00
I(h) = 5.5291235186826935, err(h) = 8.114055e-03, p* = 1.21, C = 3.245622e+00
I(h) = 5.5181629724841859, err(h) = 3.653515e-03, p* = 1.15, C = 5.845625e+00
I(h) = 5.5130846027658755, err(h) = 1.692790e-03, p* = 1.11, C = 1.083386e+01
I(h) = 5.5106832071384346, err(h) = 8.004652e-04, p* = 1.08, C = 2.049191e+01
I(h) = 5.5095304893564148, err(h) = 3.842393e-04, p* = 1.06, C = 3.934610e+01
I(h) = 5.5089709660452968, err(h) = 1.865078e-04, p* = 1.04, C = 7.639358e+01
I(h) = 5.5086971342392861, err(h) = 9.127727e-05, p* = 1.03, C = 1.495487e+02
I(h) = 5.5085623109065445, err(h) = 4.494111e-05, p* = 1.02, C = 2.945261e+02
I(h) = 5.5084956383789869, err(h) = 2.222418e-05, p* = 1.02, C = 5.825934e+02
I(h) = 5.5084625633172433, err(h) = 1.102502e-05, p* = 1.01, C = 1.156057e+03
Too large n


5.508462563317243

Видим, что не выполняются условия корректности применения метода Рунге, в частности $C=\dfrac{\Delta h}{h^p}=const$

In [163]:
ans = integ(10000) + np.pi +(np.exp(1)-1)*np.pi/2

ans

5.508463354474967

Действительно, при увеличении числа точек ответ более точен

# Задание 3 - Интеграл от осциллирующей функции

После 3 итераций метода, показанного на семинаре подинтегральная функция будет выглядеть так: $$j(x) = \dfrac{6\pi^3\sin{x}\cos{(\cos{x})}}{x(x+\pi)(x+2\pi)(x+3\pi)}$$ а искомый интеграл $$I = \dfrac{7}{8}\int_0^\pi f(x)dx+\dfrac{4}{8}\int_\pi^{2\pi} f(x)dx+\dfrac{1}{8}\int_{2\pi}^{3\pi} f(x)dx+\dfrac{1}{8}\sum_{k=0}^{\infty}\int_{\pi k}^{\pi (k+1)} j(x)dx$$ где $f(x) = \dfrac{\sin{x}\cos{(\cos{x})}}{x}$ - исходная функция

Оценим сколько слагаемых в сумме брать, чтобы погрешность не превышала $10^{-6}$: $|j(x)|\leq 6\pi^3\dfrac{1}{x^4}$, а тогда $$\dfrac{1}{8}\int_{\pi k}^{\infty}j(x)dx \leq \dfrac{6\pi^3}{8}\times \dfrac{1}{3(\pi k)^3} \leq 10^{-6}$$ Тогда $k^3 \geq 2.5*10^5$ или $k_{max}\approx 63$

Беря во внимание погрешности, связанные с приближённым вычислением интегралов, имеет смысл превысить найденное $k_{max}$, чтобы итоговая погрешность оказалась $<10^{-6}$

In [164]:
from scipy.integrate import simpson

def f(x):
    if x:
        return np.sin(x)*np.cos(np.cos(x))/x
    else:
        return np.cos(1)
    
def j(x):
    if x:
        return 6*np.pi**3*np.sin(x)*np.cos(np.cos(x))/(x*(x+np.pi)*(x+2*np.pi)*(x+3*np.pi))
    else:
        return  np.cos(1)
    
x1=np.linspace(0,np.pi, 101)
y1 = np.array([f(x) for x in x1])

I1 = 7/8*simpson(y1, dx=x1[1]-x1[0])

x10=np.linspace(0,np.pi, 101*2) # Шаг в 2 раза меньше для оценки погрешности методом Рунге
y10 = np.array([f(x) for x in x10])

I10 = 7/8*simpson(y10, dx=x10[1]-x10[0])

dI1 = abs((I1-I10)/(2**4-1)) # Так как в методе Симпсона p=4

x2=np.linspace(np.pi,2*np.pi, 101)
y2 = np.array([f(x) for x in x2])

I2 = 4/8*simpson(y2, dx=x2[1]-x2[0])

x20=np.linspace(np.pi,2*np.pi, 101*2)
y20 = np.array([f(x) for x in x20])

I20 = 4/8*simpson(y20, dx=x20[1]-x20[0])

dI2 = abs((I2-I20)/(2**4-1))

x3=np.linspace(2*np.pi,3*np.pi, 101)
y3 = np.array([f(x) for x in x3])

I3 = 1/8*simpson(y3, dx=x3[1]-x3[0])

x30=np.linspace(2*np.pi,3*np.pi, 101*10)
y30 = np.array([f(x) for x in x30])

I30 = 1/8*simpson(y30, dx=x30[1]-x30[0])

dI3 = abs((I3-I30)/(2**4-1))

x4=np.linspace(0,100*np.pi, 101*100) # взяли чуть больше чем k_max = 63
y4 = np.array([j(x) for x in x4])

I4 = 1/8*simpson(y4, dx=x4[1]-x4[0])

x40=np.linspace(0,100*np.pi, 101*100*2)
y40 = np.array([j(x) for x in x40])

I40 = 1/8*simpson(y40, dx=x40[1]-x40[0])

dI4 = abs((I4-I40)/(2**4-1))

I = I1+I2+I3+I4

dI = np.sqrt(dI1**2+dI2**2+dI3**2+dI4**2)

I,dI

(1.201969713386842, 1.9853430713995324e-10)

Ошибка приближённого вычисления интегралов значительно меньше требуемой $10^{-6}$, поэтому такой шаг интегрирования (который был использован) приемлем, и с учётом погрешности конечных пределов интегрирования (которую мы также положили $<10^{-6}$ при нахождении $k_{max}$) суммарная погрешность будет меньше заданной $10^{-6}$

Итого получили значение интеграла с требуемой точностью (не менее $10^{-6}$)

# Задание 4 - Вычисление интеграла с внутренной особенностью 

Искомый интеграл: $$\int_{-0.5}^{0.5}\dfrac{1}{\sqrt{|x|(1-x)}}dx$$

Хотим найти $\delta_1>0$ и $\delta_2>0$, чтобы $$\int_{-\delta_1}^{\delta_2}\dfrac{1}{\sqrt{|x|(1-x)}}dx < \varepsilon$$

Заметим, что при заданных пределах интегрирования $\dfrac{1}{\sqrt{1-x}}\leq\sqrt{2}$, а также можем взять $\delta_1 = \delta_2=\delta$ и воспользоваться чётностью подынтегральной функции: $$\int_{-\delta_1}^{\delta_2}\dfrac{1}{\sqrt{|x|(1-x)}}dx \leq \sqrt{2}\int_{-\delta}^{\delta}\dfrac{1}{\sqrt{|x|}}dx =2\sqrt{2}\int_{0}^{\delta}\dfrac{1}{\sqrt{x}}dx = 4\sqrt{2\delta}<\varepsilon$$

Тогда $$\delta < \dfrac{\varepsilon^2}{32}$$

# Задание 5 - Интеграл от быстроосциллирующей функции

In [165]:
def f(x,k):
    return np.exp(-x)*np.sin(k*x)

ks = [1,3,5,8,15,20,35,60,80,100,150,210,300,400,500]

x = np.linspace(0,np.pi, 500)

yks = np.array([f(x,k) for k in ks])

Iks = np.array([simpson(y, dx=x[1]-x[0]) for y in yks])

Iks_real = np.array([k/(1+k**2)-np.exp(-np.pi)*(k*np.cos(np.pi*k)+np.sin(k*np.pi))/(1+k**2) for k in ks])

Iks, Iks_real, Iks-Iks_real


(array([ 0.52160696,  0.31296418,  0.20061806,  0.11775829,  0.0692399 ,
         0.04772005,  0.02978231,  0.01594326,  0.01196106,  0.00957331,
         0.00640396,  0.00464607,  0.00359181,  0.00429288, -0.17387646]),
 array([0.52160696, 0.31296418, 0.20061806, 0.11775829, 0.06923986,
        0.04772   , 0.0297818 , 0.01594201, 0.01195796, 0.0095669 ,
        0.00637829, 0.00455602, 0.00318925, 0.00239195, 0.00191356]),
 array([-2.38684628e-11,  2.16526019e-10,  1.32128727e-09,  2.66158150e-09,
         3.99703786e-08,  4.35776331e-08,  5.12784025e-07,  1.25088505e-06,
         3.10349170e-06,  6.40707867e-06,  2.56710750e-05,  9.00537381e-05,
         4.02554306e-04,  1.90093116e-03, -1.75790020e-01]))

Видно, что при увеличении параметра $k$ (скорости осцилляции функции) ошибка в приближённом вычислении интеграла увеличивается и при достаточно больших $k$ сравнима с самим значением интеграла. Это связано с тем, что за промежуток между последовательными точками в методе Симпсона функция успевает сильно проосциллировать.

Заметим, что фактически, использовав встроенную функцию scipy.integrate.simpson, мы реализовали разбиение отрезка интегрирования $[0,\pi]$ на подотрезки и применение к каждому из них формулу Симпсона (то есть приблизили функцию её интерполяционным многочленом второй степени на каждом подотрезке) и сложили результаты для всех подотрезков, что и требовалось в задании.

Добиться требуемой точности можно, уменьшая шаг интегрирования (то есть увеличивая число подотрезков разбиения)

# Задание 6* - Интегрирование с помощью ортогональных полиномов

Преобразуем заданный интеграл: $$I = \int_0^{2\pi}\dfrac{\cos{x}dx}{\sqrt{4\pi^2-x^2}} = \int_0^{1}\dfrac{\cos{(2\pi u)} du}{\sqrt{1-u^2}}=\dfrac{1}{2}\int_{-1}^1\dfrac{\cos{(2\pi u)} du}{\sqrt{1-u^2}}$$

Видим, что в нашем случае весовая функция $w(x)=\dfrac{1}{\sqrt{1-x^2}}$, а значит мы будем использовать многочлены Чебышева 1го рода

In [166]:
from scipy.integrate import quad

def cheb_nodes(n, a=-1, b=1):
    r"""Chebyshev nodes of degree $n$ on $[a, b]$
    """

    return np.array([(a+b)/2+(b-a)/2*np.cos((2*k+1)/(2*(n))*np.pi) for k in range(n-1,-1,-1)])

def func(x, nodes, i):
    """Вычисляет подинтегральную функцию в формуле весов для квадратур Гаусса-Кристоффеля"""
    node_i = nodes[i]
    nodes_wo_i = np.delete(nodes,i)
    x_arr = np.array([x]*len(nodes_wo_i))

    return np.prod(x_arr-nodes_wo_i)/np.prod(node_i-nodes_wo_i)

def weights(nodes,a,b):
    cs = np.zeros(len(nodes))
    err = np.zeros(len(nodes))
    for i, node in enumerate(nodes):
        quads = quad(func, a,b, weight='alg', wvar=(-0.5,-0.5), args=(nodes, i))
        cs[i] = quads[0]
        err[i] = quads[1] # Созраняем погрешности, чтобы понять когда превысим машинную точность
    return cs,err

In [168]:
nodes = cheb_nodes(40)

ys = np.cos(2*np.pi*nodes)

res = weights(nodes,-1,1)

I = 1/2*np.dot(ys,res[0])

rel_err_I = np.sqrt(np.sum(np.dot(res[1]**2,ys**2)))/I 
# Относительная погрешность найденного значения, связанная с погрешностью вычисления весов с помощью quad

I,rel_err_I

(0.34601015881226427, 1.1778194850190434e-07)

Видим, что погрешность из-за приближённого вычисления весов ($\approx 10^{-8}$) значительно превышает погрешность машинного округления ($\approx 10^{-16}$). Также данная погрешность не уменьшается при увеличении числа узлов (что логично, учитывая, что весов нужно считать больше, и у каждого есть погрешность)

Тем не менее значения интеграла мы вычислили, оно согласуется с "табличным" (посчитанным Вольфрамом)